In [1]:
!nvidia-smi

Fri Jan 16 03:25:48 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Задания

    1) Подготовка данных:

      ● Реализовать программу для генерации массива случайных чисел
      (размер: 1,000,000 элементов).

    2) Оптимизация параллельного редукционного алгоритма:

      ● Реализовать редукцию суммы элементов массива с
      использованием:

        a. Только глобальной памяти.
        b. Комбинации глобальной и разделяемой памяти.

      ● Сравнить производительность и объяснить влияние использования
      разделяемой памяти.

    3) Оптимизация сортировки на GPU:

      ● Реализовать сортировку пузырьком для небольших подмассивов с
      использованием локальной памяти.

      ● Использовать глобальную память для хранения общего массива.
      
      ● Реализовать слияние отсортированных подмассивов с использованием
      разделяемой памяти.

    4) Измерение производительности:

      ● Замерить время выполнения программ с использованием разных типов
      памяти для массивов размером 10,000, 100,000 и 1,000,000 элементов.

      ● Построить графики зависимости времени выполнения от размера
      массива.

In [16]:
%%writefile practice_4.cu

#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <chrono>
#include <random>
#include <algorithm>

#define BLOCK_SIZE 256
#define SORT_SIZE 16

// --------------------------------------------------
// Task 1: Generate random array
// --------------------------------------------------
void generateRandomArray(std::vector<float>& arr, int n) {
    arr.resize(n);                                      // Resize vector
    std::random_device rd;                              // Random seed
    std::mt19937 gen(rd());                             // Generator
    std::uniform_real_distribution<float> dis(0.0f, 100.0f); // Distribution

    for (int i = 0; i < n; i++) {
        arr[i] = dis(gen);                              // Assign random value
    }
}

// --------------------------------------------------
// Task 2a: Global memory reduction
// --------------------------------------------------
__global__ void sumGlobalKernel(float* d_in, float* d_out, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        atomicAdd(d_out, d_in[tid]);
    }
}

// --------------------------------------------------
// Task 2b: Shared memory reduction
// --------------------------------------------------
__global__ void sumSharedKernel(float* d_in, float* d_out, int n) {
    __shared__ float sdata[BLOCK_SIZE];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + tid;

    sdata[tid] = (i < n) ? d_in[i] : 0.0f;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        atomicAdd(d_out, sdata[0]);
    }
}

// --------------------------------------------------
// Task 3: Bubble sort using local memory
// --------------------------------------------------
__global__ void bubbleSortLocalKernel(float* d_data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int start = tid * SORT_SIZE;

    if (start + SORT_SIZE <= n) {
        float local[SORT_SIZE];

        for (int i = 0; i < SORT_SIZE; i++) {
            local[i] = d_data[start + i];
        }

        for (int i = 0; i < SORT_SIZE - 1; i++) {
            for (int j = 0; j < SORT_SIZE - i - 1; j++) {
                if (local[j] > local[j + 1]) {
                    float tmp = local[j];
                    local[j] = local[j + 1];
                    local[j + 1] = tmp;
                }
            }
        }

        for (int i = 0; i < SORT_SIZE; i++) {
            d_data[start + i] = local[i];
        }
    }
}

// --------------------------------------------------
// Run tests for specific N
// --------------------------------------------------
void runTests(int N) {

    std::cout << "\n===================================" << std::endl;
    std::cout << "ARRAY SIZE: " << N << std::endl;
    std::cout << "===================================" << std::endl;

    std::vector<float> h_in(N);
    generateRandomArray(h_in, N);

    float *d_in, *d_out;
    cudaMalloc(&d_in, N * sizeof(float));
    cudaMalloc(&d_out, sizeof(float));

    cudaMemcpy(d_in, h_in.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // -------- Reduction: Global --------
    cudaMemset(d_out, 0, sizeof(float));
    auto s1 = std::chrono::high_resolution_clock::now();
    sumGlobalKernel<<<blocks, BLOCK_SIZE>>>(d_in, d_out, N);
    cudaDeviceSynchronize();
    auto e1 = std::chrono::high_resolution_clock::now();

    // -------- Reduction: Shared --------
    cudaMemset(d_out, 0, sizeof(float));
    auto s2 = std::chrono::high_resolution_clock::now();
    sumSharedKernel<<<blocks, BLOCK_SIZE>>>(d_in, d_out, N);
    cudaDeviceSynchronize();
    auto e2 = std::chrono::high_resolution_clock::now();

    std::cout << "[Task 2] Reduction Time:" << std::endl;
    std::cout << "  Global: "
              << std::chrono::duration<double, std::milli>(e1 - s1).count()
              << " ms" << std::endl;

    std::cout << "  Shared: "
              << std::chrono::duration<double, std::milli>(e2 - s2).count()
              << " ms" << std::endl;

    // -------- Sorting --------
    int numSub = N / SORT_SIZE;
    int threads = 128;
    int blocksSort = (numSub + threads - 1) / threads;

    auto s3 = std::chrono::high_resolution_clock::now();
    bubbleSortLocalKernel<<<blocksSort, threads>>>(d_in, N);
    cudaDeviceSynchronize();
    auto e3 = std::chrono::high_resolution_clock::now();

    std::cout << "[Task 3] Bubble Sort Time: "
              << std::chrono::duration<double, std::milli>(e3 - s3).count()
              << " ms" << std::endl;

    cudaFree(d_in);
    cudaFree(d_out);
}

// --------------------------------------------------
int main() {
    int sizes[] = {10000, 100000, 1000000};

    std::cout << "PRACTICAL WORK #4" << std::endl;

    for (int n : sizes) {
        runTests(n);
    }

    return 0;
}

Writing practice_4.cu


In [17]:
!nvcc practice_4.cu -o main

In [18]:
!./main

PRACTICAL WORK #4

ARRAY SIZE: 10000
[Task 2] Reduction Time:
  Global: 12.3189 ms
  Shared: 0.005347 ms
[Task 3] Bubble Sort Time: 0.0029 ms

ARRAY SIZE: 100000
[Task 2] Reduction Time:
  Global: 0.039668 ms
  Shared: 0.005495 ms
[Task 3] Bubble Sort Time: 0.002254 ms

ARRAY SIZE: 1000000
[Task 2] Reduction Time:
  Global: 0.0696 ms
  Shared: 0.00538 ms
[Task 3] Bubble Sort Time: 0.002184 ms
